In [146]:

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

import csv
import sys, os
sys.path.append('~/dsi/capstones/cap_3/')

# from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, DateType
# import pyspark as ps

# import scipy as stats
import re
import dateparser
import datetime
import math
# import json

# import researchpy as rp
# import statsmodels.api as sm
# from statsmodels.formula.api import ols

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

In [147]:
# CLEANING - Cast NaN/Null
def cast_nan(step_df):

    make_nan = ['No', 'N/A', 'No cellular connectivity', '-']

    df_obj = step_df.select_dtypes(['object'])

    step_df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

    step_df.replace(to_replace=make_nan, value=np.nan, inplace=True)
    
    return step_df

In [278]:
# CLEANING - Drop Words
useless_drop_words = [
    'launch_status',
    'misc_colors',
    'misc_models',
    'misc_sar_eu',
    'misc_sar',
    'sound_alert_types',
    'features_clock',
    'features_alarm',
    'features_languages',
    'selfie_camera_v1',
    'main_camera',
    'selfie_camera_triple',
    'main_camera_v1',
    'memory'
]

apple_drop_words = [
    'body_keyboard',
    'memory_phonebook',
    'memory_call_records',
    'features_messaging',
    'features_games',
    'features_java'
]

analysis_drop_words = [
    'body_dimensions',
    'memory_internal',
    'battery_talk_time',
    'network_speed',
    'battery_stand-by',
    'tests_performance',
    'tests_display',
    'tests_camera',
    'tests_loudspeaker',
    'tests_audio_quality',
    'tests_battery_life',
    'battery_music_play',
    'selfie_camera',
    'features_browser',
    'comms_infrared_port',
#     'selfie_camera_dual',
    'camera',
    'main_camera_dual_or_triple'
]

In [149]:
# CLEAING - Date Parser
def parse_date(date_str):
    
    # Parsing variables
    years = range(1994, 2017)

    months = ['January', 'February', 'March', 'April', 'May', 'June', 
            'July', 'August', 'September', 'October', 'November', 'December']

    months_short = ['Jan-', 'Feb-', 'Mar-', 'Apr-', 'May-', 'Jun-', 
                    'Jul-', 'Aug-', 'Sep-', 'Oct-', 'Nov-', 'Dec-']

    months_num = np.linspace(0.0, 1.0, num=12)

    months_int = range(1, 12)

    months_dict = {
        'January':1, 
        'February':2, 
        'March':3, 
        'April':4, 
        'May':5, 
        'June':6, 
        'July':7, 
        'August':8, 
        'September':9, 
        'October':10, 
        'November':11, 
        'December':12
    }

    quarters = ['Q1', '1Q', 'Q2', '2Q', 'Q3', '3Q', 'Q4', '4Q']

    quarters_dict = {
        'Q1': 'February',
        '1Q': 'February',
        'Q2': 'May',
        '2Q': 'May',
        '3Q': 'August',
        'Q3': 'August',
        '4Q': 'November',
        'Q4': 'November'
    }

    # Removes preceding and proceding spaces
    date_str = date_str.strip()
    
    # Changes strings with "?" to "remove"
    if '?' in date_str:
        date_str = 'remove'
    
    # Removes errant release dates
    if '.' in date_str:
        split_str = date_str.split('.')
        date_str = split_str[0]
    
    # Changes Quarter to Middle Month of Quarter
    for key, val in quarters_dict.items():
        if key in date_str:
            split_2 = date_str.split()
            split_2[split_2.index(key)] = val
            date_str = " ".join(split_2)

    # Converts Year Only to Year + ' July'
    if len(date_str) < 5 and len(date_str) > 1:
        date_str += ' July'    
    
    # If date is formated in short form i.e. "Feb-01"
    m_index = 0
    for m in months_short:
        if m in date_str:
            split_3 = date_str.split('-')
            split_3[0] = months[m_index]
            split_3[1] = f'20{split_3[1]}'
            date_str = " ".join(split_3)
        m_index += 1
       
    # Date Parsing str using module dataparser    
    date_tm = dateparser.parse(date_str)
    if isinstance(date_tm, datetime.date):
        date_tm = date_tm - datetime.timedelta(days=date_tm.day-1)
           
    return date_tm

In [150]:
# CLEANING - Execute Date Parser
def refactor_time(df, col='launch_announced'):
    step_df = df[df['launch_announced'].notna()]

    step_df['launch_announced'] = step_df['launch_announced'].apply(parse_date)
    step_df['launch_announced'] = step_df['launch_announced'].dt.to_period('M')
    # step_df.loc[df['launch_announced'] >= '2006']
    
    return step_df

In [151]:
# CLEANING - Parse Price
def clean_price(price_value): 

#     price_words = ['About']
#     currencies = ['USD', 'EUR', 'INR']

    if type(price_value)!=str:
        if math.isnan(price_value):
            return np.nan
        
    elif price_value == '-':
        return np.nan
    
    elif price_value == 'About BTC 0.15/ETH 4.78(crypto curr)':
        return 800.0
    
    else:
        price_value = price_value.split(' ')

        if price_value[2] == 'EUR':
            return float(price_value[1]) / 1.2
        elif price_value[2] == 'INR':
            return float(price_value[1]) * 60
        elif price_value[2] == 'USD':
            return float(price_value[1])

In [152]:
# CLEANING - Parse Video 
def clean_video(video_camera_value):
    
    if type(video_camera_value)!=str:
        if math.isnan(video_camera_value):
            return np.nan
    
    if '@' in video_camera_value:
        return 'Yes'
    
    elif 'x' in video_camera_value:
        video_camera_value = re.split('x|@', video_camera_value)
        video_camera_value = int(video_camera_value[0]) * int(video_camera_value[1])
    
    elif video_camera_value != np.nan:
        video_camera_value.replace('w', '')
        video_camera_value = video_camera_value.split('p')[0]
        
        if 'yes' in video_camera_value.lower():
            video_camera_value = 'Yes'
    
    return video_camera_value

In [153]:
'''
# Cleaning - Screen Size

def screen_refactor(s, split_char=' '):
    s = str(s)
    s = s.replace('| -', '')
    s = s.replace('|', '')
    s = s.strip()
    
    
    if isinstance(s, str):
        split_s = s.split(split_char)
        # try 
        return split_s[0]
    else:
        return s
    
period_df['screen_in'] = period_df['display_size'].apply(screen_refactor)
'''

"\n# Cleaning - Screen Size\n\ndef screen_refactor(s, split_char=' '):\n    s = str(s)\n    s = s.replace('| -', '')\n    s = s.replace('|', '')\n    s = s.strip()\n    \n    \n    if isinstance(s, str):\n        split_s = s.split(split_char)\n        # try \n        return split_s[0]\n    else:\n        return s\n    \nperiod_df['screen_in'] = period_df['display_size'].apply(screen_refactor)\n"

In [154]:

# EXTRACTING - Extract Device Type
tablet_words = ['tab', 'tablet', 'pad', 'book']
watch_words = ['watch', 'gear', 'fit', 'band']

def parse_type(model_string, words_list):

    for check_word in words_list:
        if check_word in model_string.lower():
            return True
        else:
            return False

In [155]:
# EXTRACTION - Features General Function
def extract_features(org_col, data_frame, new_cols, category, 
                     regex_1 = None, regex_2 = None):
    
    for n in new_cols:
        col_str = str(category) + '_' + n.replace(' ', '_')
        data_frame[col_str] = False
    
    idx = 0
    for row in data_frame[str(org_col)]:
        if type(row)!=str:
            if math.isnan(row):
                pass
        else:
            for n in new_cols:
                if n.lower() in row.lower():
                    col_str = str(category) + '_' + n.replace(' ', '_')
                    data_frame[col_str].iloc[idx] = True

        idx += 1

## Extraction Lists

In [156]:
# EXTRACTING - Sensors
relevant_sensors = ['accelerometer', 'gyro', 'heart rate', 'fingerprint', 'compass',
                    'proximity', 'barometer', 'spo2', 'iris scanner', 'gesture', 
                    'tempurature', 'altimeter', 'infrared face recognition'
                   ]

In [157]:
# Will need to troubleshoot !
relevant_body = ['water resistant', 'waterproof', 'water proof', 'splash', 'pay', 'stylus', 
                 'kickstand', 'flashlight']

In [158]:
# May need to troubleshoot !
relevant_display_type = ['LCD', 'OLED', 'AMOLED', 'TFT', 'STN', 'CSTN', 'ASV', 'IPS',
                        'resistive', 'capacitive', 'touchscreen']

In [159]:
# Add Single Core After !
relevant_platform_cpu = ['octa-core', 'hexa-core', 'quad-core', 'dual-core']

In [213]:
relevant_sound_loudspeaker = ['stereo']

In [160]:
relevant_comms_wlan = ['a/', 'b/', '/g', '/i', '/n', '/ac', '/ax', 
                       'dual-band', 'hotspot', 'DLNA', 'Wi-Fi Direct']

In [161]:
relevant_comms_bluetooth = ['1.1', '1.2', '1.5', '2.0', '2.1', '2.2', '3.0', '3.1', 
                      '4.0', '4.1', '4.2', '5.0', '5.1', 'A2DP', 'EDR', 'LE', 'aptX']

In [162]:
relevant_comms_gps = ['GLONASS', 'A-GPS', 'GALILEO', 'BDS', 'QZSS']

In [163]:
# Will need to troubleshoot !
relevant_battery = ['removable', 'non-removable', 'li-ion', 'li-po']

In [164]:
relevant_main_camera_features = ['flash', 'HDR', 'panorama']

In [165]:
relevant_sound = ['active noise cancelation', 'dedicated mic', 'HDR']

In [166]:
relevant_selfie_camera_features = ['flash', 'HDR']

In [167]:
relevant_battery_charging = ['fast', 'wireless', 'reverse']

In [168]:
relevant_display = ['home button', '3D']

In [169]:
# Combine 'glass back' and 'back glass' !
relevant_body_build = ['plastic back', 'glass back', 'back glass', 'ceramic back']

In [247]:
# Will need to troubleshoot
relevant_main_camera = ['wide', 'ultrawide', 'telephoto', 'zoom', 
                        'depth', 'laser', 'ois', 'pdaf']

In [208]:
relevant_network_technology = ['GSM', 'CDMA', 'HSPA', 'EDVO', 'LTE', 'UMTS']

In [209]:
relevant_network = ['TD-SCDMA', 'HSDPA']

## Execute

In [171]:

df_org = pd.read_csv('../data/gsm.csv')
df = df_org.copy()

In [172]:
df = cast_nan(df)

In [173]:
df.drop(columns=useless_drop_words, inplace=True)
df.drop(columns=apple_drop_words, inplace=True)
df.drop(columns=analysis_drop_words, inplace=True)

In [174]:
df = refactor_time(df)

In [175]:
df['misc_price'].apply(lambda x: clean_price(x))

0       275.000000
1       141.666667
2       208.333333
3       100.000000
4       191.666667
5       191.666667
6              NaN
7       166.666667
8        91.666667
9       291.666667
10      183.333333
11       91.666667
12       75.000000
13             NaN
14      166.666667
15             NaN
16      125.000000
17             NaN
18             NaN
19      208.333333
20      150.000000
21      166.666667
22      108.333333
23       75.000000
24       66.666667
25      108.333333
26      166.666667
27      125.000000
28      216.666667
29      150.000000
           ...    
8950           NaN
8951           NaN
8952           NaN
8953           NaN
8954           NaN
8955           NaN
8956           NaN
8957           NaN
8958     50.000000
8959           NaN
8960    150.000000
8961           NaN
8962           NaN
8963           NaN
8964           NaN
8965           NaN
8966           NaN
8967           NaN
8968           NaN
8969           NaN
8970           NaN
8971        

In [176]:
df['main_camera_video'] = df['main_camera_video'].apply(clean_video)

In [177]:
df['is_tablet'] = df['model'].apply(lambda x: parse_type(x, tablet_words))
df['is_watch'] = df['model'].apply(lambda x: parse_type(x, watch_words))


In [178]:
extract_features(org_col = 'features_sensors', 
                 data_frame = df, 
                 new_cols = relevant_sensors, 
                 category = 'sensor'
                )

In [179]:
extract_features(org_col = 'body', 
                 data_frame = df, 
                 new_cols = relevant_body, 
                 category = 'body'
                )

df['body_waterproof'] = df[['body_waterproof', 'body_water_proof']].any(axis=1)
df['body_water_resistant'] = df[['body_water_resistant', 'body_splash']].any(axis=1)

df.drop(columns=['body_water_proof', 'body_splash'], inplace=True)


In [180]:
# df

In [182]:
extract_features(org_col = 'display_type', 
                 data_frame = df, 
                 new_cols = relevant_display_type, 
                 category = 'display_type'
                )

In [183]:
# df

In [185]:
extract_features(org_col = 'platform_cpu', 
                 data_frame = df, 
                 new_cols = relevant_platform_cpu, 
                 category = 'platform_cpu'
                )


In [245]:
df.loc[(df['platform_cpu'].notnull()) & 
       (df['platform_cpu_octa-core'] == False) &
       (df['platform_cpu_hexa-core'] == False) &
       (df['platform_cpu_quad-core'] == False) &
       (df['platform_cpu_dual-core'] == False), 
       'platform_cpu_single-core'] = True
df['platform_cpu_single-core'].fillna(False, inplace=True)

In [248]:
df.head(5)

,oem,model,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,body_weight,body_sim,body,display_type,display_size,display_resolution,platform_os,platform_chipset,platform_cpu,memory_card_slot,main_camera_single,main_camera_video,selfie_camera_single,selfie_camera_video,sound_loudspeaker,sound_3.5mm_jack,comms_wlan,comms_bluetooth,comms_gps,comms_radio,comms_usb,features_sensors,battery,misc_price,network_3g_bands,network_4g_bands,network,platform_gpu,main_camera_features,sound,features,selfie_camera_features,display_protection,battery_charging,comms_nfc,display,main_camera_dual,selfie_camera_dual,body_build,camera,memory,main_camera_triple,main_camera_quad,main_camera_dual_or_triple,main_camera_five,is_tablet,is_watch,sensor_accelerometer,sensor_gyro,sensor_heart_rate,sensor_fingerprint,sensor_compass,sensor_proximity,sensor_barometer,sensor_spo2,sensor_iris_scanner,sensor_gesture,sensor_tempurature,sensor_altimeter,sensor_infrared_face_recognition,body_water_resistant,body_waterproof,body_pay,body_stylus,body_kickstand,body_flashlight,display_type_LCD,display_type_OLED,display_type_AMOLED,display_type_TFT,display_type_STN,display_type_CSTN,display_type_ASV,display_type_IPS,display_type_resistive,display_type_capacitive,display_type_touchscreen,platform_cpu_octa-core,platform_cpu_hexa-core,platform_cpu_quad-core,platform_cpu_dual-core,comms_wlan_a/,comms_wlan_b/,comms_wlan_/g,comms_wlan_/i,comms_wlan_/n,comms_wlan_/ac,comms_wlan_/ax,comms_wlan_dual-band,comms_wlan_hotspot,comms_wlan_DLNA,comms_wlan_Wi-Fi_Direct,comms_bluetooth_1.1,comms_bluetooth_1.2,comms_bluetooth_1.5,comms_bluetooth_2.0,comms_bluetooth_2.1,comms_bluetooth_2.2,comms_bluetooth_3.0,comms_bluetooth_3.1,comms_bluetooth_4.0,comms_bluetooth_4.1,comms_bluetooth_4.2,comms_bluetooth_5.0,comms_bluetooth_5.1,comms_bluetooth_A2DP,comms_bluetooth_EDR,comms_bluetooth_LE,comms_bluetooth_aptX,comms_gps_GLONASS,comms_gps_A-GPS,comms_gps_GALILEO,comms_gps_BDS,comms_gps_QZSS,battery_removable,battery_non-removable,battery_li-ion,battery_li-po,main_camera_features_flash,main_camera_features_HDR,main_camera_features_panorama,sound_active_noise_cancelation,sound_dedicated_mic,sound_HDR,selfie_camera_features_flash,selfie_camera_features_HDR,battery_charging_fast,battery_charging_wireless,battery_charging_reverse,display_home_button,display_3D,body_build_plastic_back,body_build_glass_back,body_build_ceramic_back,main_camera_single_wide,main_camera_single_ultrawide,main_camera_single_telephoto,main_camera_single_zoom,main_camera_double_wide,main_camera_double_ultrawide,main_camera_double_telephoto,main_camera_double_zoom,main_camera_dual_wide,main_camera_dual_ultrawide,main_camera_dual_telephoto,main_camera_dual_zoom,main_camera_triple_wide,main_camera_triple_ultrawide,main_camera_triple_telephoto,main_camera_triple_zoom,main_camera_quad_wide,main_camera_quad_ultrawide,main_camera_quad_telephoto,main_camera_quad_zoom,network_technology_GSM,network_technology_CDMA,network_technology_HSPA,network_technology_EDVO,network_technology_LTE,network_technology_UMTS,network_TD-SCDMA,network_HSDPA,sound_loudspeaker_stereo,platform_cpu_single-core
0,Acer,Chromebook Tab 10,NaN,NaN,NaN,NaN,2018-03,544.3 g (1.20 lb),NaN,Stylus,"IPS LCD capacitive touchscreen, 16M colors","9.7 inches, 291.4 cm (~71.0% screen-to-body ra...","1536 x 2048 pixels, 4:3 ratio (~264 ppi density)",Chrome OS,Rockchip RK3399,Hexa-core (4x Cortex-A53 & 2x Cortex-A72),"microSD, up to 128 GB",5 MP,720,2 MP,NaN,"Yes, with stereo speakers",Yes,"Wi-Fi 802.11 b/g/n/ac, dual-band, WiFi Direct","4.1, A2DP, LE",NaN,NaN,"3.1, Type-C 1.0 reversible connector",Accelerometer,Non-removable Li-Po 4500 mAh battery (34 Wh),About 330 EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,True,True,False,True,Fa

In [214]:
extract_features(org_col = 'sound_loudspeaker', 
                 data_frame = df, 
                 new_cols = relevant_sound_loudspeaker, 
                 category = 'sound_loudspeaker'
                )

In [186]:
extract_features(org_col = 'comms_wlan', 
                 data_frame = df, 
                 new_cols = relevant_comms_wlan, 
                 category = 'comms_wlan'
                )

In [279]:
df.rename(columns={'comms_wlan_a/':'comms_wlan_a', 
                   'comms_wlan_b/':'comms_wlan_b', 
                   'comms_wlan_/g':'comms_wlan_g', 
                   'comms_wlan_/i':'comms_wlan_i', 
                   'comms_wlan_/n':'comms_wlan_n',
                   'comms_wlan_/ac':'comms_wlan_ac', 
                   'comms_wlan_/ax':'comms_wlan_ax'}, inplace=True)

In [192]:
extract_features(org_col = 'comms_bluetooth', 
                 data_frame = df, 
                 new_cols = relevant_comms_bluetooth, 
                 category = 'comms_bluetooth'
                )

In [193]:
extract_features(org_col = 'comms_gps', 
                 data_frame = df, 
                 new_cols = relevant_comms_gps, 
                 category = 'comms_gps'
                )

In [194]:
extract_features(org_col = 'battery', 
                 data_frame = df, 
                 new_cols = relevant_battery, 
                 category = 'battery'
                )



In [234]:
df.loc[(df['battery_non-removable'] == True) & 
       (df['battery_removable'] == True), 'battery_removable'] = False


In [195]:
extract_features(org_col = 'main_camera_features', 
                 data_frame = df, 
                 new_cols = relevant_main_camera_features, 
                 category = 'main_camera_features'
                )

In [196]:
extract_features(org_col = 'sound', 
                 data_frame = df, 
                 new_cols = relevant_sound, 
                 category = 'sound'
                )

In [197]:
extract_features(org_col = 'selfie_camera_features', 
                 data_frame = df, 
                 new_cols = relevant_selfie_camera_features, 
                 category = 'selfie_camera_features'
                )

In [198]:
extract_features(org_col = 'battery_charging', 
                 data_frame = df, 
                 new_cols = relevant_battery_charging, 
                 category = 'battery_charging'
                )

In [199]:
extract_features(org_col = 'display', 
                 data_frame = df, 
                 new_cols = relevant_display, 
                 category = 'display'
                )

In [200]:
extract_features(org_col = 'body_build', 
                 data_frame = df, 
                 new_cols = relevant_body_build, 
                 category = 'body_build'
                )


In [201]:
df['body_build_glass_back'] = df[['body_build_glass_back', 'body_build_back_glass']].any(axis=1)

df.drop(columns=['body_build_back_glass'], inplace=True)

In [249]:
extract_features(org_col = 'main_camera_single', 
                 data_frame = df, 
                 new_cols = relevant_main_camera, 
                 category = 'main_camera_single'
                )

In [250]:
extract_features(org_col = 'main_camera_dual', 
                 data_frame = df, 
                 new_cols = relevant_main_camera, 
                 category = 'main_camera_dual'
                )

In [251]:
extract_features(org_col = 'main_camera_triple', 
                 data_frame = df, 
                 new_cols = relevant_main_camera, 
                 category = 'main_camera_triple'
                )

In [252]:
extract_features(org_col = 'main_camera_quad', 
                 data_frame = df, 
                 new_cols = relevant_main_camera, 
                 category = 'main_camera_quad'
                )

In [253]:
extract_features(org_col = 'network_technology', 
                 data_frame = df, 
                 new_cols = relevant_network_technology, 
                 category = 'network_technology'
                )

In [211]:
extract_features(org_col = 'network', 
                 data_frame = df, 
                 new_cols = relevant_network, 
                 category = 'network'
                )

In [256]:
df.loc[(df['main_camera_single_wide'] == True) |
       (df['main_camera_dual_wide'] == True) |
       (df['main_camera_triple_wide'] == True) |
       (df['main_camera_quad_wide'] == True), 
       'camera_feature_wide'] = True

df.drop(columns = ['main_camera_single_wide', 'main_camera_dual_wide', 
                   'main_camera_triple_wide', 'main_camera_quad_wide'], inplace=True)

In [258]:
df.loc[(df['main_camera_single_ultrawide'] == True) |
       (df['main_camera_dual_ultrawide'] == True) |
       (df['main_camera_triple_ultrawide'] == True) |
       (df['main_camera_quad_ultrawide'] == True), 
       'camera_feature_ultrawide'] = True

df.drop(columns = ['main_camera_single_ultrawide', 'main_camera_dual_ultrawide', 
                   'main_camera_triple_ultrawide', 'main_camera_quad_ultrawide'], inplace=True)

In [259]:
df.loc[(df['main_camera_single_telephoto'] == True) |
       (df['main_camera_dual_telephoto'] == True) |
       (df['main_camera_triple_telephoto'] == True) |
       (df['main_camera_quad_telephoto'] == True), 
       'camera_feature_telephoto'] = True

df.drop(columns = ['main_camera_single_telephoto', 'main_camera_dual_telephoto', 
                   'main_camera_triple_telephoto', 'main_camera_quad_telephoto'], inplace=True)

In [261]:
df.loc[(df['main_camera_single_zoom'] == True) |
       (df['main_camera_dual_zoom'] == True) |
       (df['main_camera_triple_zoom'] == True) |
       (df['main_camera_quad_zoom'] == True), 
       'camera_feature_zoom'] = True

df.drop(columns = ['main_camera_single_zoom', 'main_camera_dual_zoom', 
                   'main_camera_triple_zoom', 'main_camera_quad_zoom'], inplace=True)

KeyError: 'main_camera_single_zoom'

In [262]:
df.loc[(df['main_camera_single_depth'] == True) |
       (df['main_camera_dual_depth'] == True) |
       (df['main_camera_triple_depth'] == True) |
       (df['main_camera_quad_depth'] == True), 
       'camera_feature_depth'] = True

df.drop(columns = ['main_camera_single_depth', 'main_camera_dual_depth', 
                   'main_camera_triple_depth', 'main_camera_quad_depth'], inplace=True)

In [263]:
df.loc[(df['main_camera_single_laser'] == True) |
       (df['main_camera_dual_laser'] == True) |
       (df['main_camera_triple_laser'] == True) |
       (df['main_camera_quad_laser'] == True), 
       'camera_feature_laser'] = True

df.drop(columns = ['main_camera_single_laser', 'main_camera_dual_laser', 
                   'main_camera_triple_laser', 'main_camera_quad_laser'], inplace=True)

In [264]:
df.loc[(df['main_camera_single_ois'] == True) |
       (df['main_camera_dual_ois'] == True) |
       (df['main_camera_triple_ois'] == True) |
       (df['main_camera_quad_ois'] == True), 
       'camera_feature_ois'] = True

df.drop(columns = ['main_camera_single_ois', 'main_camera_dual_ois', 
                   'main_camera_triple_ois', 'main_camera_quad_ois'], inplace=True)

In [265]:
df.loc[(df['main_camera_single_pdaf'] == True) |
       (df['main_camera_dual_pdaf'] == True) |
       (df['main_camera_triple_pdaf'] == True) |
       (df['main_camera_quad_pdaf'] == True), 
       'camera_feature_pdaf'] = True

df.drop(columns = ['main_camera_single_pdaf', 'main_camera_dual_pdaf', 
                   'main_camera_triple_pdaf', 'main_camera_quad_pdaf'], inplace=True)

In [269]:
df.drop(columns = ['main_camera_double_wide', 'main_camera_double_ultrawide', 
                   'main_camera_double_telephoto', 'main_camera_double_zoom'], inplace=True)

In [275]:
df['camera_feature_wide'].fillna(False, inplace=True) 
df['camera_feature_ultrawide'].fillna(False, inplace=True)
df['camera_feature_telephoto'].fillna(False, inplace=True)
df['camera_feature_zoom'].fillna(False, inplace=True)
df['camera_feature_depth'].fillna(False, inplace=True)
df['camera_feature_laser'].fillna(False, inplace=True)
df['camera_feature_ois'].fillna(False, inplace=True)
df['camera_feature_pdaf'].fillna(False, inplace=True)


In [276]:
df

,oem,model,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,body_weight,body_sim,body,display_type,display_size,display_resolution,platform_os,platform_chipset,platform_cpu,memory_card_slot,main_camera_single,main_camera_video,selfie_camera_single,selfie_camera_video,sound_loudspeaker,sound_3.5mm_jack,comms_wlan,comms_bluetooth,comms_gps,comms_radio,comms_usb,features_sensors,battery,misc_price,network_3g_bands,network_4g_bands,network,platform_gpu,main_camera_features,sound,features,selfie_camera_features,display_protection,battery_charging,comms_nfc,display,main_camera_dual,selfie_camera_dual,body_build,camera,memory,main_camera_triple,main_camera_quad,main_camera_dual_or_triple,main_camera_five,is_tablet,is_watch,sensor_accelerometer,sensor_gyro,sensor_heart_rate,sensor_fingerprint,sensor_compass,sensor_proximity,sensor_barometer,sensor_spo2,sensor_iris_scanner,sensor_gesture,sensor_tempurature,sensor_altimeter,sensor_infrared_face_recognition,body_water_resistant,body_waterproof,body_pay,body_stylus,body_kickstand,body_flashlight,display_type_LCD,display_type_OLED,display_type_AMOLED,display_type_TFT,display_type_STN,display_type_CSTN,display_type_ASV,display_type_IPS,display_type_resistive,display_type_capacitive,display_type_touchscreen,platform_cpu_octa-core,platform_cpu_hexa-core,platform_cpu_quad-core,platform_cpu_dual-core,comms_wlan_a/,comms_wlan_b/,comms_wlan_/g,comms_wlan_/i,comms_wlan_/n,comms_wlan_/ac,comms_wlan_/ax,comms_wlan_dual-band,comms_wlan_hotspot,comms_wlan_DLNA,comms_wlan_Wi-Fi_Direct,comms_bluetooth_1.1,comms_bluetooth_1.2,comms_bluetooth_1.5,comms_bluetooth_2.0,comms_bluetooth_2.1,comms_bluetooth_2.2,comms_bluetooth_3.0,comms_bluetooth_3.1,comms_bluetooth_4.0,comms_bluetooth_4.1,comms_bluetooth_4.2,comms_bluetooth_5.0,comms_bluetooth_5.1,comms_bluetooth_A2DP,comms_bluetooth_EDR,comms_bluetooth_LE,comms_bluetooth_aptX,comms_gps_GLONASS,comms_gps_A-GPS,comms_gps_GALILEO,comms_gps_BDS,comms_gps_QZSS,battery_removable,battery_non-removable,battery_li-ion,battery_li-po,main_camera_features_flash,main_camera_features_HDR,main_camera_features_panorama,sound_active_noise_cancelation,sound_dedicated_mic,sound_HDR,selfie_camera_features_flash,selfie_camera_features_HDR,battery_charging_fast,battery_charging_wireless,battery_charging_reverse,display_home_button,display_3D,body_build_plastic_back,body_build_glass_back,body_build_ceramic_back,network_technology_GSM,network_technology_CDMA,network_technology_HSPA,network_technology_EDVO,network_technology_LTE,network_technology_UMTS,network_TD-SCDMA,network_HSDPA,sound_loudspeaker_stereo,platform_cpu_single-core,camera_feature_wide,camera_feature_ultrawide,camera_feature_telephoto,camera_feature_zoom,camera_feature_depth,camera_feature_laser,camera_feature_ois,camera_feature_pdaf
0,Acer,Chromebook Tab 10,NaN,NaN,NaN,NaN,2018-03,544.3 g (1.20 lb),NaN,Stylus,"IPS LCD capacitive touchscreen, 16M colors","9.7 inches, 291.4 cm (~71.0% screen-to-body ra...","1536 x 2048 pixels, 4:3 ratio (~264 ppi density)",Chrome OS,Rockchip RK3399,Hexa-core (4x Cortex-A53 & 2x Cortex-A72),"microSD, up to 128 GB",5 MP,720,2 MP,NaN,"Yes, with stereo speakers",Yes,"Wi-Fi 802.11 b/g/n/ac, dual-band, WiFi Direct","4.1, A2DP, LE",NaN,NaN,"3.1, Type-C 1.0 reversible connector",Accelerometer,Non-removable Li-Po 4500 mAh battery (34 Wh),About 330 EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,True,True,False,True,False,False,False,True,True,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,True,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,F

In [280]:
df.to_csv('../results/output.csv')